# Variational autoencoder
This notebook will investiga

In [1]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import seaborn as sns
from collections import defaultdict

from models.variational_autoencoder import VariationalAutoencoder
from models.autoencoder import Autoencoder
from models.continuous_bernoulli_loss import continuous_bernoulli_loss

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from visual.latent_plane_mosaic import LatentPlaneMosaic
from visual.latent_interpolation_mosaic import LatentInterpolationMosaic
from visual.sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [2]:
model_df = pd.DataFrame(data = {
    'path' : ["models\\saved\\ae", "models\\saved\\vae_10", "models\\saved\\vae_100" , "models\\saved\\vae_1000"],
    'ae_type' : ['ae', 'vae', 'vae', 'vae'],
    },
    index = ['ae', 'vae_10', 'vae_100', 'vae_1000'],
)


In [3]:
models = []
for name, model in model_df.iterrows():
    if model['ae_type'] == "ae":
        models.append(keras.models.load_model(model['path'], 
                         custom_objects={"Autoencoder": Autoencoder,
                                         "continuous_bernoulli_loss": continuous_bernoulli_loss}))
    elif model['ae_type'] == "vae":
        models.append(keras.models.load_model(model['path'], 
                                 custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                                 "continuous_bernoulli_loss":  continuous_bernoulli_loss}))
model_df['model'] = models    

In [4]:
num_models = model_df.shape[0]
latent_dim = 10

In [5]:
model_df.head()

,path,ae_type,model
ae,models\saved\ae,ae,<models.autoencoder.Autoencoder object at 0x00...
vae_10,models\saved\vae_10,vae,<models.variational_autoencoder.VariationalAut...
vae_100,models\saved\vae_100,vae,<models.variational_autoencoder.VariationalAut...
vae_1000,models\saved\vae_1000,vae,<models.variational_autoencoder.VariationalAut...


## Preprocess data


In [6]:
(_, _), (test_digits, test_labels) = keras.datasets.mnist.load_data()
test_digits = np.expand_dims(test_digits, -1).astype("float32") / 255
input_shape = test_digits.shape[1:]
num_test = test_labels.shape[0]

## Load Variational autoencoder

### Print setup details

## Reconstruct and dimension reductions

In [9]:
# Fill
latent_df_list = []
dimension_df_list = []


for name, model in model_df.iterrows():
    if model['ae_type'] == "ae":
        code = model['model'].encoder(test_digits).numpy()
        code_std = np.empty(shape = code.shape)*np.nan
        
    elif model['ae_type'] == "vae":
        digit_distribution = model['model'].encoder(test_digits).numpy()
        code = digit_distribution[:,0,:]
        code_std = np.sqrt(np.exp(digit_distribution[:,1,:]))
    
    code_distance = np.linalg.norm(code, axis = 1)
    code_radius = np.power(np.prod(code_std, axis = 1),1/latent_dim)
    code_loss = tf.reduce_mean(tf.reduce_mean(tf.keras.losses.binary_crossentropy(test_digits, model['model'](test_digits)),axis = -1),axis = -1).numpy()
    code_tsne = TSNE(n_components = 2).fit_transform(code)
    
    _latent_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : test_labels,
        'loss' : code_loss,
        'distance' : code_distance,
        'radius' : code_radius,
    })
    #    'tsne_0' : code_tsne[:,0],
    #    'tsne_1' : code_tsne[:,1],
    #})
    latent_df_list.append(_latent_df)   
    
    dimension_labels = test_labels.repeat(repeats = latent_dim)
    dimension_digit_index = np.arange(num_test).repeat(repeats = latent_dim)
    dimension_dim_index = np.tile(np.arange(latent_dim), num_test)
    dimension_value = code.reshape(-1)
    dimension_std = code_std.reshape(-1)
    
    _dimension_df = pd.DataFrame(data = {
        'model_name' : name,
        'label' : dimension_labels,
        'digit_index' : dimension_digit_index,
        'dim_index' : dimension_dim_index,
        'value' : dimension_value,
        'std' : dimension_std
    })
    
    dimension_df_list.append(_dimension_df)

        
latent_df = pd.concat(latent_df_list)
dimension_df = pd.concat(dimension_df_list)
dimension_mean_df = dimension_df.groupby(by = ['model_name', 'dim_index'],as_index = False).mean()
dimension_std_df  = dimension_df.groupby(by = ['model_name', 'dim_index'],as_index = False).std()


## Investigation

### Show reconstructions

In [10]:
latent_df.head()

,model_name,label,loss,distance,radius
0,ae,7,0.070490,15.820858,NaN
1,ae,2,0.117689,9.960896,NaN
2,ae,1,0.038200,24.145765,NaN
3,ae,0,0.096915,8.166095,NaN
4,ae,4,0.084998,14.815680,NaN


In [11]:
dimension_df.head()

,model_name,label,digit_index,dim_index,value,std
0,ae,7,0,0,8.920861,NaN
1,ae,7,0,1,2.935239,NaN
2,ae,7,0,2,0.101040,NaN
3,ae,7,0,3,3.835251,NaN
4,ae,7,0,4,5.167331,NaN


In [12]:
dimension_mean_df.head()

,model_name,dim_index,label,digit_index,value,std
0,ae,0,4.4434,4999.5,-1.309089,NaN
1,ae,1,4.4434,4999.5,0.659992,NaN
2,ae,2,4.4434,4999.5,1.053212,NaN
3,ae,3,4.4434,4999.5,3.870928,NaN
4,ae,4,4.4434,4999.5,2.164852,NaN


In [13]:
dimension_std_df.head()

,model_name,dim_index,label,digit_index,value,std
0,ae,0,2.895865,2886.89568,3.059542,NaN
1,ae,1,2.895865,2886.89568,3.437961,NaN
2,ae,2,2.895865,2886.89568,2.503023,NaN
3,ae,3,2.895865,2886.89568,2.846906,NaN
4,ae,4,2.895865,2886.89568,2.707609,NaN


In [103]:
num_col = 4
num_row = 3
num_img = num_row*num_col
rec_index = rnd.randint(num_test,size =(num_row*num_col,) )
digits = test_digits
digits = digits[rec_index]

In [104]:

model = model_df.loc['ae','model']

reconstructions = model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        pair = np.concatenate((digits[i], reconstructions[i]), axis = 1)
        if row == 0: 
            ver_img = pair
        else:
            ver_img = np.concatenate((ver_img,
                                 pair), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)

sns.heatmap(img[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

In [105]:

num_dig = 14
rec_index = rnd.randint(num_test,size =(num_dig,) )
digits = test_digits
digits = digits[rec_index]

reconstructions = []
for name, model in model_df.iterrows():
    reconstructions.append(model['model'](digits))

for dig_i in range(num_dig):
    rec_im = digits[dig_i]
    model_i = 0
    for name, model in model_df.iterrows():
        
        rec_im = np.concatenate((rec_im,
                         reconstructions[model_i][dig_i]),
                         axis = 0)
        model_i = model_i + 1
    if dig_i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis = 1)
    
sns.heatmap(im[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

### Latent

In [106]:

indeces = rnd.randint(num_test,size = (3,))
indeces = [1,2,3]
ul = test_digits[indeces[0]]
ur = test_digits[indeces[1]]
dl = test_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)


In [107]:

model = model_df.loc['vae_10','model']
mosaic = LatentInterpolationMosaic(
                          model.encode,
                          model.decoder,
                          test_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic



sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

<AxesSubplot:>

### Scatter

In [108]:
model_name = 'vae_1000'
scatter = latent_df[latent_df['model_name']==model_name][['tsne_0', 'tsne_1']].to_numpy()
SampleScatterGUI(scatter, test_labels, test_digits)


In [109]:
model_name = "vae_1000"
sns.jointplot(data = latent_df[latent_df['model_name'] == model_name ], x = 'tsne_0', y = 'tsne_1',
              hue = 'label',
              palette = 'colorblind')

In [110]:
model_name = "vae_1000"
sns.relplot(data = latent_df[latent_df['model_name'] == model_name ], x = 'tsne_0', y = 'tsne_1',
              hue = 'label',
              palette = 'colorblind',
           kind = 'scatter')

### Spread

Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [111]:
axis_index = [0,-1]
scaling_factors = np.array([2,2])

In [112]:
model_name = 'ae'
model = model_df.loc[model_name, 'model']
latent_vectors = model_df.loc[model_name, 'latent_base'][axis_index]
latent_origin  = model_df.loc[model_name, 'latent_mean']
mosaic = LatentPlaneMosaic(model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

#fig,ax = plt.subplots(1,1)
#ax.imshow(mosaic)
sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

KeyError: 'latent_base'

## Spred 

In [14]:
sns.displot(data = latent_df, x = 'loss', hue = 'model_name', kind = 'kde')

In [15]:
sns.displot(data = latent_df, x = 'distance',hue = 'model_name', kind = 'kde')

In [18]:
sns.displot(data = latent_df, x = 'radius',hue = 'model_name', kind = 'kde')

C:\Users\joelw\anaconda3\lib\site-packages\seaborn\distributions.py:306: UserWarning: Dataset has 0 variance; skipping density estimate.
  warnings.warn(msg, UserWarning)


In [17]:
######### IMPORTANT ########
sns.relplot(x = dimension_mean_df['std'], y = dimension_std_df['value'], hue = dimension_mean_df['model_name'])

In [61]:
sns.relplot(data = dimension_df,
           x = 'value',
           y = 'std',
           col = 'dim_index',
           col_wrap = 4,
           hue = 'model_name')

## Generative power

In [66]:
model_name = 'ae'
mean = np.zeros(latent_dim)
mean = model_df.loc[model_name,'latent_mean']
cov  = 1*np.eye(latent_dim)
num_col = 15
num_row = 10
num_img = num_row*num_col
latent_points = rnd.multivariate_normal(mean, cov, size = (num_img))



In [67]:
model = model_df.loc[model_name, 'model']
title = "Reconstruction with autoencoder"
fig, ax = plt.subplots()
reconstructions = model.decoder(latent_points)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        if row == 0: 
            ver_img = reconstructions[i]
        else:
            ver_img = np.concatenate((ver_img,
                                 reconstructions[i]), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)
sns.heatmap(img[:,:,0])



<AxesSubplot:>

In [68]:
mean = np.zeros(latent_dim)
cov  = 1*np.eye(latent_dim)
num_dig = 5
latent_points = rnd.multivariate_normal(mean, cov, size = (num_dig))

reconstructions = []
for name, model in model_df.iterrows():
    reconstructions.append(model['model'].decoder(latent_points))
    
fig, axs = plt.subplots(num_models, num_dig, sharex = True, sharey = True)
for model_i in range(num_models):
    for dig_i in range(num_dig):
        ax = axs[model_i, dig_i]
        sns.heatmap(reconstructions[model_i][dig_i][:,:,0], ax = ax)



array([ 0.04270896, -0.02646526, -0.01551382,  0.00895875,  0.00756909,
       -0.00550758, -0.01186571,  0.00023721, -0.0082873 ,  0.0526603 ],
      dtype=float32)